In [1]:
from gurobipy import *
import snap
from graphviz import Digraph
import pandas as pd
import numpy as np

In [2]:
# G1 = snap.TUNGraph.New()
# G1.AddNode(0)
# G1.AddNode(1)
# G1.AddNode(2)
# G1.AddNode(3)
# G1.AddNode(4)
# G1.AddNode(5)
# G1.AddNode(6)
# G1.AddNode(7)
# G1.AddNode(8)
# G1.AddNode(9)
# G1.AddNode(10)
# G1.AddNode(11)
# G1.AddNode(12)
# G1.AddNode(13)
# G1.AddNode(14)
# G1.AddEdge(0,1)
# G1.AddEdge(0,2)
# G1.AddEdge(2,3)
# G1.AddEdge(1,3)
# G1.AddEdge(3,4)
# G1.AddEdge(4,5)
# G1.AddEdge(4,6)
# G1.AddEdge(0,7)
# G1.AddEdge(7,8)
# G1.AddEdge(7,9)
# G1.AddEdge(9,10)
# G1.AddEdge(0,11)
# G1.AddEdge(11,12)
# G1.AddEdge(11,13)
# G1.AddEdge(13,14)
# G1

In [3]:
# G1 = snap.GenRndGnm(snap.TUNGraph, 360, 900)
G1 = snap.LoadEdgeList(snap.TUNGraph, "graph.txt", 0, 1)

In [4]:
#參數設定
Budget = 2
init_S = 0

In [5]:
# output 圖片 !!!!Data大容易跑很久!!!!
# labels = {}
# for NI in G1.Nodes():
#         labels[NI.GetId()] = str(NI.GetId())
# G1.DrawGViz(snap.gvlDot, "graphIP_LP_relaxLP2.png", " ", labels)

In [6]:
#刪除孤單的點
lonely_node = []
for NI in G1.Nodes():
    if NI.GetOutDeg()==0:
        lonely_node.append(NI.GetId())
        G1.DelNode(NI.GetId())

print("lonely_node = ",lonely_node)

lonely_node =  []


In [7]:
AllnodeID = []
InfectednodeID = [init_S]
V = [] #AllnodeID\0(s) #vulnerablenodeID
vaccinatednodeID = [] #vaccinated的node
for NI in G1.Nodes():
    AllnodeID.append(NI.GetId())
    if(NI.GetId()!=init_S):
        V.append(NI.GetId())

In [8]:
# AllnodeID = [] #全部的node
# InfectednodeID = [0]
# vulnerablenodeID = []
# vaccinatednodeID = [] #vaccinated的node
# # inf_plus_vul_minus_zero = []
# for NI in G1.Nodes():
#     AllnodeID.append(NI.GetId())
#     if(NI.GetId()!=0):
#         vulnerablenodeID.append(NI.GetId())
#     print("node: %d" % NI.GetId())

In [9]:
layer = 0
for NI in G1.Nodes():
    if(G1.GetShortPath(NI.GetId(),init_S)>layer):
        layer = G1.GetShortPath(NI.GetId(),init_S)
layer

13

# IP Programming

In [10]:
B = Budget #budget
S = init_S #初始感染點
#Create empty model
IP_model = Model('IP Vaccination Strategy')

#add variablies
#S = model.addVars([(u,t) for u in V for t in range(1,layer+1)],vtype=GRB.BINARY, name="S")
X = IP_model.addVars([(u,t) for u in V for t in range(1,layer+1) ],vtype=GRB.BINARY, name="X")
Y = IP_model.addVars([(v) for v in V],vtype=GRB.BINARY, name="Y")
IP_model.update()
#add objective function
# obj = 0
# for v in V:
#     obj = obj+Y[v]
IP_model.setObjective(quicksum(Y[v] for v in V) ,GRB.MAXIMIZE)

#constraint 1
IP_model.addConstrs(quicksum(X[u,t] for u in V)<=B for t in range(1,layer+1))
#constraint 2
#model.addConstrs(quicksum(X[u,t] for t in range(1,layer+1))<=1 for u in V) 
#constraint 3
IP_model.addConstrs(quicksum(X[u,t] for u in V for t in range(1,(G1.GetShortPath(S,v)-G1.GetShortPath(u,v))+1) if t>=0 )>= Y[v]  for v in V)
IP_model.update()
IP_model.optimize()

Academic license - for non-commercial use only - expires 2021-07-24
Using license file C:\Users\user\gurobi.lic
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 3904 rows, 54474 columns and 9070668 nonzeros
Model fingerprint: 0x7572a910
Variable types: 0 continuous, 54474 integer (54474 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 0 rows and 38238 columns (presolve time = 5s) ...
Presolve removed 4 rows and 43358 columns (presolve time = 10s) ...
Presolve removed 191 rows and 43904 columns
Presolve time: 12.39s
Presolved: 3713 rows, 10570 columns, 1197094 nonzeros
Variable types: 0 continuous, 10570 integer (10381 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.    

In [11]:
print("IP Programming")
print("保護的個數=",IP_model.ObjVal)
for var in IP_model.getVars():
    if(var.x>0):
        print(var.varName,'\t', var.x)

IP Programming
保護的個數= 3889.0
X[14,1] 	 1.0
X[2543,1] 	 1.0
X[180,2] 	 1.0
X[3403,4] 	 1.0
X[886,7] 	 1.0
X[46,3] 	 1.0
X[372,3] 	 1.0
X[1073,2] 	 1.0
X[734,4] 	 1.0
X[3417,9] 	 1.0
X[3693,5] 	 1.0
X[2495,8] 	 1.0
X[1215,9] 	 1.0
X[1357,8] 	 1.0
X[2465,6] 	 1.0
X[2914,6] 	 1.0
X[3079,5] 	 1.0
X[2453,7] 	 1.0
Y[14] 	 1.0
Y[2543] 	 1.0
Y[2013] 	 1.0
Y[2714] 	 1.0
Y[180] 	 1.0
Y[2082] 	 1.0
Y[2938] 	 1.0
Y[3016] 	 1.0
Y[1] 	 1.0
Y[300] 	 1.0
Y[815] 	 1.0
Y[942] 	 1.0
Y[952] 	 1.0
Y[1009] 	 1.0
Y[1033] 	 1.0
Y[1171] 	 1.0
Y[1465] 	 1.0
Y[1603] 	 1.0
Y[1856] 	 1.0
Y[2069] 	 1.0
Y[2080] 	 1.0
Y[2717] 	 1.0
Y[3678] 	 1.0
Y[3799] 	 1.0
Y[3185] 	 1.0
Y[3245] 	 1.0
Y[2455] 	 1.0
Y[2682] 	 1.0
Y[2751] 	 1.0
Y[3138] 	 1.0
Y[3351] 	 1.0
Y[2954] 	 1.0
Y[3839] 	 1.0
Y[2285] 	 1.0
Y[2305] 	 1.0
Y[2976] 	 1.0
Y[3274] 	 1.0
Y[3403] 	 1.0
Y[3436] 	 1.0
Y[3843] 	 1.0
Y[3845] 	 1.0
Y[886] 	 1.0
Y[1048] 	 1.0
Y[1054] 	 1.0
Y[1082] 	 1.0
Y[1562] 	 1.0
Y[2154] 	 1.0
Y[3443] 	 1.0
Y[3642] 	 1.0
Y[1214] 	 1.0
Y[

Y[946] 	 1.0
Y[1046] 	 1.0
Y[2277] 	 1.0
Y[3537] 	 1.0
Y[3697] 	 1.0
Y[3035] 	 1.0
Y[33] 	 1.0
Y[62] 	 1.0
Y[70] 	 1.0
Y[456] 	 1.0
Y[635] 	 1.0
Y[3493] 	 1.0
Y[3508] 	 1.0
Y[3763] 	 1.0
Y[281] 	 1.0
Y[1504] 	 1.0
Y[1803] 	 1.0
Y[1983] 	 1.0
Y[2269] 	 1.0
Y[2975] 	 1.0
Y[3011] 	 1.0
Y[559] 	 1.0
Y[2883] 	 1.0
Y[34] 	 1.0
Y[936] 	 1.0
Y[3714] 	 1.0
Y[1631] 	 1.0
Y[1689] 	 1.0
Y[35] 	 1.0
Y[1516] 	 1.0
Y[2597] 	 1.0
Y[2872] 	 1.0
Y[3014] 	 1.0
Y[1765] 	 1.0
Y[2429] 	 1.0
Y[2755] 	 1.0
Y[3323] 	 1.0
Y[3696] 	 1.0
Y[3318] 	 1.0
Y[3428] 	 1.0
Y[3596] 	 1.0
Y[3606] 	 1.0
Y[3195] 	 1.0
Y[3613] 	 1.0
Y[3590] 	 1.0
Y[38] 	 1.0
Y[2261] 	 1.0
Y[2591] 	 1.0
Y[253] 	 1.0
Y[492] 	 1.0
Y[876] 	 1.0
Y[1127] 	 1.0
Y[1162] 	 1.0
Y[1251] 	 1.0
Y[1280] 	 1.0
Y[1571] 	 1.0
Y[1576] 	 1.0
Y[1694] 	 1.0
Y[2238] 	 1.0
Y[2720] 	 1.0
Y[3306] 	 1.0
Y[3335] 	 1.0
Y[3630] 	 1.0
Y[3690] 	 1.0
Y[3729] 	 1.0
Y[2891] 	 1.0
Y[3103] 	 1.0
Y[700] 	 1.0
Y[812] 	 1.0
Y[959] 	 1.0
Y[1325] 	 1.0
Y[1382] 	 1.0
Y[3434] 	 1.0
Y[

Y[3077] 	 1.0
Y[84] 	 1.0
Y[85] 	 1.0
Y[2797] 	 1.0
Y[3263] 	 1.0
Y[3299] 	 1.0
Y[3362] 	 1.0
Y[3372] 	 1.0
Y[3801] 	 1.0
Y[87] 	 1.0
Y[89] 	 1.0
Y[2508] 	 1.0
Y[3033] 	 1.0
Y[90] 	 1.0
Y[806] 	 1.0
Y[1115] 	 1.0
Y[1658] 	 1.0
Y[1963] 	 1.0
Y[2852] 	 1.0
Y[2414] 	 1.0
Y[91] 	 1.0
Y[113] 	 1.0
Y[889] 	 1.0
Y[1455] 	 1.0
Y[1903] 	 1.0
Y[2003] 	 1.0
Y[2710] 	 1.0
Y[3158] 	 1.0
Y[3576] 	 1.0
Y[3654] 	 1.0
Y[2415] 	 1.0
Y[1806] 	 1.0
Y[2313] 	 1.0
Y[2989] 	 1.0
Y[1329] 	 1.0
Y[1377] 	 1.0
Y[1437] 	 1.0
Y[1441] 	 1.0
Y[2707] 	 1.0
Y[3647] 	 1.0
Y[3871] 	 1.0
Y[93] 	 1.0
Y[1727] 	 1.0
Y[2117] 	 1.0
Y[2209] 	 1.0
Y[910] 	 1.0
Y[1402] 	 1.0
Y[94] 	 1.0
Y[2706] 	 1.0
Y[1552] 	 1.0
Y[1662] 	 1.0
Y[1770] 	 1.0
Y[2044] 	 1.0
Y[2431] 	 1.0
Y[3581] 	 1.0
Y[3859] 	 1.0
Y[96] 	 1.0
Y[537] 	 1.0
Y[706] 	 1.0
Y[1152] 	 1.0
Y[1229] 	 1.0
Y[1428] 	 1.0
Y[1684] 	 1.0
Y[1968] 	 1.0
Y[3276] 	 1.0
Y[3629] 	 1.0
Y[716] 	 1.0
Y[1716] 	 1.0
Y[1742] 	 1.0
Y[1822] 	 1.0
Y[2609] 	 1.0
Y[1804] 	 1.0
Y[2358] 	 1.0
Y[2

Y[257] 	 1.0
Y[1994] 	 1.0
Y[2762] 	 1.0
Y[3031] 	 1.0
Y[3371] 	 1.0
Y[2630] 	 1.0
Y[2141] 	 1.0
Y[2744] 	 1.0
Y[3608] 	 1.0
Y[3361] 	 1.0
Y[3862] 	 1.0
Y[263] 	 1.0
Y[2385] 	 1.0
Y[2537] 	 1.0
Y[2628] 	 1.0
Y[465] 	 1.0
Y[558] 	 1.0
Y[1697] 	 1.0
Y[1753] 	 1.0
Y[2780] 	 1.0
Y[3475] 	 1.0
Y[3783] 	 1.0
Y[267] 	 1.0
Y[1239] 	 1.0
Y[1799] 	 1.0
Y[2096] 	 1.0
Y[2550] 	 1.0
Y[3840] 	 1.0
Y[1364] 	 1.0
Y[1160] 	 1.0
Y[483] 	 1.0
Y[1102] 	 1.0
Y[414] 	 1.0
Y[3633] 	 1.0
Y[271] 	 1.0
Y[1277] 	 1.0
Y[1283] 	 1.0
Y[3030] 	 1.0
Y[3719] 	 1.0
Y[1386] 	 1.0
Y[3535] 	 1.0
Y[1470] 	 1.0
Y[2540] 	 1.0
Y[3239] 	 1.0
Y[2803] 	 1.0
Y[1438] 	 1.0
Y[2046] 	 1.0
Y[2809] 	 1.0
Y[1034] 	 1.0
Y[274] 	 1.0
Y[1174] 	 1.0
Y[1380] 	 1.0
Y[1480] 	 1.0
Y[2033] 	 1.0
Y[2849] 	 1.0
Y[2800] 	 1.0
Y[276] 	 1.0
Y[1471] 	 1.0
Y[278] 	 1.0
Y[1191] 	 1.0
Y[1549] 	 1.0
Y[2530] 	 1.0
Y[1776] 	 1.0
Y[2252] 	 1.0
Y[2680] 	 1.0
Y[279] 	 1.0
Y[1847] 	 1.0
Y[1949] 	 1.0
Y[2137] 	 1.0
Y[2263] 	 1.0
Y[2452] 	 1.0
Y[3770] 	 1.0
Y[18

Y[624] 	 1.0
Y[2384] 	 1.0
Y[563] 	 1.0
Y[2946] 	 1.0
Y[489] 	 1.0
Y[565] 	 1.0
Y[602] 	 1.0
Y[3171] 	 1.0
Y[570] 	 1.0
Y[1876] 	 1.0
Y[702] 	 1.0
Y[2098] 	 1.0
Y[1849] 	 1.0
Y[572] 	 1.0
Y[960] 	 1.0
Y[1088] 	 1.0
Y[1126] 	 1.0
Y[3833] 	 1.0
Y[2963] 	 1.0
Y[3426] 	 1.0
Y[3779] 	 1.0
Y[867] 	 1.0
Y[1012] 	 1.0
Y[574] 	 1.0
Y[357] 	 1.0
Y[577] 	 1.0
Y[579] 	 1.0
Y[2733] 	 1.0
Y[2901] 	 1.0
Y[3612] 	 1.0
Y[2054] 	 1.0
Y[2153] 	 1.0
Y[580] 	 1.0
Y[1137] 	 1.0
Y[2473] 	 1.0
Y[3377] 	 1.0
Y[2195] 	 1.0
Y[2851] 	 1.0
Y[582] 	 1.0
Y[583] 	 1.0
Y[1818] 	 1.0
Y[586] 	 1.0
Y[588] 	 1.0
Y[589] 	 1.0
Y[3500] 	 1.0
Y[593] 	 1.0
Y[1069] 	 1.0
Y[594] 	 1.0
Y[2482] 	 1.0
Y[3887] 	 1.0
Y[595] 	 1.0
Y[601] 	 1.0
Y[1226] 	 1.0
Y[1383] 	 1.0
Y[1764] 	 1.0
Y[1953] 	 1.0
Y[2904] 	 1.0
Y[3856] 	 1.0
Y[2624] 	 1.0
Y[210] 	 1.0
Y[2711] 	 1.0
Y[2723] 	 1.0
Y[608] 	 1.0
Y[609] 	 1.0
Y[610] 	 1.0
Y[2026] 	 1.0
Y[611] 	 1.0
Y[2411] 	 1.0
Y[612] 	 1.0
Y[1141] 	 1.0
Y[1607] 	 1.0
Y[2324] 	 1.0
Y[3634] 	 1.0
Y[534] 	

Y[385] 	 1.0
Y[698] 	 1.0
Y[1359] 	 1.0
Y[1814] 	 1.0
Y[2560] 	 1.0
Y[1703] 	 1.0
Y[1153] 	 1.0
Y[1717] 	 1.0
Y[1724] 	 1.0
Y[2576] 	 1.0
Y[1740] 	 1.0
Y[1755] 	 1.0
Y[1763] 	 1.0
Y[1769] 	 1.0
Y[1959] 	 1.0
Y[2861] 	 1.0
Y[3414] 	 1.0
Y[1778] 	 1.0
Y[2813] 	 1.0
Y[3562] 	 1.0
Y[2262] 	 1.0
Y[1808] 	 1.0
Y[1810] 	 1.0
Y[1811] 	 1.0
Y[1812] 	 1.0
Y[2311] 	 1.0
Y[1821] 	 1.0
Y[1826] 	 1.0
Y[750] 	 1.0
Y[1845] 	 1.0
Y[1860] 	 1.0
Y[1867] 	 1.0
Y[2183] 	 1.0
Y[1881] 	 1.0
Y[3805] 	 1.0
Y[1882] 	 1.0
Y[1885] 	 1.0
Y[3319] 	 1.0
Y[1894] 	 1.0
Y[2328] 	 1.0
Y[2676] 	 1.0
Y[1895] 	 1.0
Y[1899] 	 1.0
Y[1901] 	 1.0
Y[1906] 	 1.0
Y[242] 	 1.0
Y[2387] 	 1.0
Y[353] 	 1.0
Y[1545] 	 1.0
Y[1938] 	 1.0
Y[1942] 	 1.0
Y[1943] 	 1.0
Y[437] 	 1.0
Y[1951] 	 1.0
Y[473] 	 1.0
Y[1683] 	 1.0
Y[1679] 	 1.0
Y[1988] 	 1.0
Y[262] 	 1.0
Y[3032] 	 1.0
Y[2007] 	 1.0
Y[2870] 	 1.0
Y[3625] 	 1.0
Y[2021] 	 1.0
Y[973] 	 1.0
Y[2245] 	 1.0
Y[2045] 	 1.0
Y[2265] 	 1.0
Y[2465] 	 1.0
Y[2914] 	 1.0
Y[2090] 	 1.0
Y[2063] 	 1.0
Y

# LP Programming

In [12]:
B = Budget #budget
S = init_S #初始感染點
#Create empty model
LP_model = Model('LP Vaccination Strategy')

#add variablies
#S = model.addVars([(u,t) for u in V for t in range(1,layer+1)],vtype=GRB.BINARY, name="S")
X = LP_model.addVars([(u,t) for u in V for t in range(1,layer+1) ],lb=0,ub=1, name="X")
Y = LP_model.addVars([(v) for v in V],lb=0,ub=1, name="Y")
LP_model.update()
#add objective function
# obj = 0
# for v in V:
#     obj = obj+Y[v]
LP_model.setObjective(quicksum(Y[v] for v in V) ,GRB.MAXIMIZE)

#constraint 1
LP_model.addConstrs(quicksum(X[u,t] for u in V)<=B for t in range(1,layer+1))
#constraint 3
LP_model.addConstrs(quicksum(X[u,t] for u in V for t in range(1,(G1.GetShortPath(S,v)-G1.GetShortPath(u,v))+1) if t>=0 )>= Y[v]  for v in V)
LP_model.update()
LP_model.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 3904 rows, 54474 columns and 9070668 nonzeros
Model fingerprint: 0xb5f2599a
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 4 rows and 32836 columns
Presolve time: 4.60s
Presolved: 3900 rows, 21638 columns, 6816348 nonzeros


Barrier performed 0 iterations in 5.95 seconds
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex
Solved in 34 iterations and 5.97 seconds
Optimal objective  3.889000000e+03


In [13]:
THE_X = [] #取出X[,]
X_Pro = [] #取出X[,]的機率
print("LP Programming")
print("保護的個數=",LP_model.ObjVal)
for var in LP_model.getVars():
    if(var.x>0):
        if(var.varName[0]=="X"):
            THE_X.append(var.varName) 
            X_Pro.append(var.x)
        print(var.varName,'\t', var.x)

LP Programming
保護的個數= 3889.0
X[14,1] 	 0.5
X[1744,1] 	 0.5
X[2543,1] 	 1.0
X[180,2] 	 0.5
X[2082,2] 	 0.5
X[2938,2] 	 0.5
X[1,6] 	 1.0
X[815,6] 	 1.0
X[3138,5] 	 1.0
X[2285,5] 	 1.0
X[886,7] 	 1.0
X[3443,7] 	 1.0
X[46,3] 	 1.0
X[372,3] 	 1.0
X[1073,2] 	 0.5
X[734,4] 	 1.0
X[1038,4] 	 1.0
X[4,8] 	 1.0
X[667,8] 	 1.0
X[18,9] 	 1.0
X[2089,9] 	 1.0
X[3841,10] 	 1.0
X[3317,10] 	 1.0
X[3089,11] 	 1.0
X[1544,11] 	 1.0
Y[14] 	 0.5
Y[1744] 	 0.5
Y[2543] 	 1.0
Y[2013] 	 1.0
Y[2714] 	 1.0
Y[180] 	 1.0
Y[2082] 	 1.0
Y[2938] 	 1.0
Y[3016] 	 1.0
Y[1] 	 1.0
Y[300] 	 1.0
Y[815] 	 1.0
Y[942] 	 1.0
Y[952] 	 1.0
Y[1009] 	 1.0
Y[1033] 	 1.0
Y[1171] 	 1.0
Y[1465] 	 1.0
Y[1603] 	 1.0
Y[1856] 	 1.0
Y[2069] 	 1.0
Y[2080] 	 1.0
Y[2717] 	 1.0
Y[3678] 	 1.0
Y[3799] 	 1.0
Y[3185] 	 1.0
Y[3245] 	 1.0
Y[2455] 	 1.0
Y[2682] 	 1.0
Y[2751] 	 1.0
Y[3138] 	 1.0
Y[3351] 	 1.0
Y[2954] 	 1.0
Y[3839] 	 1.0
Y[2285] 	 1.0
Y[2305] 	 1.0
Y[2976] 	 1.0
Y[3274] 	 1.0
Y[3403] 	 1.0
Y[3436] 	 1.0
Y[3843] 	 1.0
Y[3845] 	 1.0
Y[886] 

Y[2774] 	 1.0
Y[2931] 	 1.0
Y[3333] 	 1.0
Y[3368] 	 1.0
Y[18] 	 1.0
Y[51] 	 1.0
Y[1004] 	 1.0
Y[2329] 	 1.0
Y[2646] 	 1.0
Y[3017] 	 1.0
Y[767] 	 1.0
Y[804] 	 1.0
Y[1456] 	 1.0
Y[1832] 	 1.0
Y[2089] 	 1.0
Y[2131] 	 1.0
Y[2333] 	 1.0
Y[2623] 	 1.0
Y[3040] 	 1.0
Y[3044] 	 1.0
Y[3321] 	 1.0
Y[3663] 	 1.0
Y[3666] 	 1.0
Y[3740] 	 1.0
Y[2752] 	 1.0
Y[3481] 	 1.0
Y[2671] 	 1.0
Y[19] 	 1.0
Y[190] 	 1.0
Y[375] 	 1.0
Y[2323] 	 1.0
Y[2639] 	 1.0
Y[1130] 	 1.0
Y[1381] 	 1.0
Y[3061] 	 1.0
Y[3068] 	 1.0
Y[3793] 	 1.0
Y[2832] 	 1.0
Y[20] 	 1.0
Y[36] 	 1.0
Y[530] 	 1.0
Y[657] 	 1.0
Y[693] 	 1.0
Y[974] 	 1.0
Y[1403] 	 1.0
Y[1962] 	 1.0
Y[2750] 	 1.0
Y[1485] 	 1.0
Y[2399] 	 1.0
Y[3421] 	 1.0
Y[843] 	 1.0
Y[3720] 	 1.0
Y[1789] 	 1.0
Y[1875] 	 1.0
Y[2192] 	 1.0
Y[3234] 	 1.0
Y[3269] 	 1.0
Y[3848] 	 1.0
Y[2767] 	 1.0
Y[3621] 	 1.0
Y[3842] 	 1.0
Y[2856] 	 1.0
Y[22] 	 1.0
Y[1125] 	 1.0
Y[2611] 	 1.0
Y[1426] 	 1.0
Y[1519] 	 1.0
Y[1842] 	 1.0
Y[1976] 	 1.0
Y[2310] 	 1.0
Y[2583] 	 1.0
Y[2670] 	 1.0
Y[2709] 	 1.0

Y[1757] 	 1.0
Y[68] 	 1.0
Y[2625] 	 1.0
Y[3317] 	 1.0
Y[69] 	 1.0
Y[71] 	 1.0
Y[1888] 	 1.0
Y[2247] 	 1.0
Y[2806] 	 1.0
Y[2302] 	 1.0
Y[2314] 	 1.0
Y[2814] 	 1.0
Y[3140] 	 1.0
Y[3230] 	 1.0
Y[3449] 	 1.0
Y[3492] 	 1.0
Y[3600] 	 1.0
Y[3653] 	 1.0
Y[3741] 	 1.0
Y[3265] 	 1.0
Y[1274] 	 1.0
Y[1330] 	 1.0
Y[1771] 	 1.0
Y[2787] 	 1.0
Y[3491] 	 1.0
Y[74] 	 1.0
Y[840] 	 1.0
Y[1105] 	 1.0
Y[1412] 	 1.0
Y[3003] 	 1.0
Y[3778] 	 1.0
Y[2219] 	 1.0
Y[3256] 	 1.0
Y[457] 	 1.0
Y[869] 	 1.0
Y[1533] 	 1.0
Y[2864] 	 1.0
Y[1820] 	 1.0
Y[2085] 	 1.0
Y[2283] 	 1.0
Y[3386] 	 1.0
Y[3558] 	 1.0
Y[3704] 	 1.0
Y[3882] 	 1.0
Y[925] 	 1.0
Y[1231] 	 1.0
Y[10] 	 1.0
Y[307] 	 1.0
Y[374] 	 1.0
Y[496] 	 1.0
Y[2185] 	 1.0
Y[2433] 	 1.0
Y[3710] 	 1.0
Y[1221] 	 1.0
Y[1493] 	 1.0
Y[1846] 	 1.0
Y[3408] 	 1.0
Y[3400] 	 1.0
Y[3098] 	 1.0
Y[3136] 	 1.0
Y[3433] 	 1.0
Y[2536] 	 1.0
Y[2922] 	 1.0
Y[1728] 	 1.0
Y[547] 	 1.0
Y[909] 	 1.0
Y[981] 	 1.0
Y[1536] 	 1.0
Y[1786] 	 1.0
Y[2992] 	 1.0
Y[3553] 	 1.0
Y[3865] 	 1.0
Y[2792] 	 1.

Y[144] 	 1.0
Y[1780] 	 1.0
Y[1924] 	 1.0
Y[147] 	 1.0
Y[825] 	 1.0
Y[2030] 	 1.0
Y[148] 	 1.0
Y[3192] 	 1.0
Y[1937] 	 1.0
Y[2081] 	 1.0
Y[2369] 	 1.0
Y[2698] 	 1.0
Y[3610] 	 1.0
Y[149] 	 1.0
Y[454] 	 1.0
Y[2840] 	 1.0
Y[1292] 	 1.0
Y[150] 	 1.0
Y[1584] 	 1.0
Y[2061] 	 1.0
Y[3499] 	 1.0
Y[3638] 	 1.0
Y[1469] 	 1.0
Y[1995] 	 1.0
Y[2215] 	 1.0
Y[704] 	 1.0
Y[1520] 	 1.0
Y[151] 	 1.0
Y[2253] 	 1.0
Y[3039] 	 1.0
Y[1523] 	 1.0
Y[153] 	 1.0
Y[427] 	 1.0
Y[567] 	 1.0
Y[699] 	 1.0
Y[800] 	 1.0
Y[1223] 	 1.0
Y[2988] 	 1.0
Y[3133] 	 1.0
Y[3506] 	 1.0
Y[3855] 	 1.0
Y[3329] 	 1.0
Y[3357] 	 1.0
Y[3627] 	 1.0
Y[1926] 	 1.0
Y[39] 	 1.0
Y[154] 	 1.0
Y[2338] 	 1.0
Y[2377] 	 1.0
Y[2568] 	 1.0
Y[2926] 	 1.0
Y[156] 	 1.0
Y[159] 	 1.0
Y[445] 	 1.0
Y[692] 	 1.0
Y[1902] 	 1.0
Y[2145] 	 1.0
Y[2748] 	 1.0
Y[528] 	 1.0
Y[3657] 	 1.0
Y[306] 	 1.0
Y[317] 	 1.0
Y[571] 	 1.0
Y[915] 	 1.0
Y[968] 	 1.0
Y[1019] 	 1.0
Y[1734] 	 1.0
Y[1824] 	 1.0
Y[1759] 	 1.0
Y[161] 	 1.0
Y[269] 	 1.0
Y[442] 	 1.0
Y[1029] 	 1.0
Y[1589] 

Y[729] 	 1.0
Y[3151] 	 1.0
Y[3175] 	 1.0
Y[329] 	 1.0
Y[330] 	 1.0
Y[331] 	 1.0
Y[1395] 	 1.0
Y[332] 	 1.0
Y[1555] 	 1.0
Y[1896] 	 1.0
Y[2076] 	 1.0
Y[2287] 	 1.0
Y[2396] 	 1.0
Y[2566] 	 1.0
Y[3197] 	 1.0
Y[2088] 	 1.0
Y[2653] 	 1.0
Y[3280] 	 1.0
Y[2467] 	 1.0
Y[2971] 	 1.0
Y[3168] 	 1.0
Y[3354] 	 1.0
Y[3637] 	 1.0
Y[3712] 	 1.0
Y[334] 	 1.0
Y[335] 	 1.0
Y[552] 	 1.0
Y[336] 	 1.0
Y[337] 	 1.0
Y[216] 	 1.0
Y[2091] 	 1.0
Y[338] 	 1.0
Y[341] 	 1.0
Y[1163] 	 1.0
Y[343] 	 1.0
Y[518] 	 1.0
Y[346] 	 1.0
Y[347] 	 1.0
Y[1269] 	 1.0
Y[2168] 	 1.0
Y[2343] 	 1.0
Y[2722] 	 1.0
Y[2940] 	 1.0
Y[3849] 	 1.0
Y[2749] 	 1.0
Y[3092] 	 1.0
Y[2104] 	 1.0
Y[348] 	 1.0
Y[1293] 	 1.0
Y[349] 	 1.0
Y[356] 	 1.0
Y[462] 	 1.0
Y[821] 	 1.0
Y[912] 	 1.0
Y[1027] 	 1.0
Y[350] 	 1.0
Y[351] 	 1.0
Y[1351] 	 1.0
Y[2441] 	 1.0
Y[3447] 	 1.0
Y[3835] 	 1.0
Y[3851] 	 1.0
Y[2448] 	 1.0
Y[2519] 	 1.0
Y[2605] 	 1.0
Y[3872] 	 1.0
Y[1647] 	 1.0
Y[352] 	 1.0
Y[1928] 	 1.0
Y[358] 	 1.0
Y[451] 	 1.0
Y[1354] 	 1.0
Y[359] 	 1.0
Y[1331]

Y[1235] 	 1.0
Y[3542] 	 1.0
Y[3505] 	 1.0
Y[1248] 	 1.0
Y[1249] 	 1.0
Y[1250] 	 1.0
Y[175] 	 1.0
Y[2542] 	 1.0
Y[3173] 	 1.0
Y[3628] 	 1.0
Y[2073] 	 1.0
Y[1259] 	 1.0
Y[1260] 	 1.0
Y[1266] 	 1.0
Y[1732] 	 1.0
Y[1273] 	 1.0
Y[2906] 	 1.0
Y[3111] 	 1.0
Y[1307] 	 1.0
Y[1308] 	 1.0
Y[2526] 	 1.0
Y[1309] 	 1.0
Y[3125] 	 1.0
Y[1313] 	 1.0
Y[1316] 	 1.0
Y[824] 	 1.0
Y[1318] 	 1.0
Y[2531] 	 1.0
Y[1319] 	 1.0
Y[1320] 	 1.0
Y[2111] 	 1.0
Y[1321] 	 1.0
Y[1323] 	 1.0
Y[1447] 	 1.0
Y[2805] 	 1.0
Y[2900] 	 1.0
Y[3624] 	 1.0
Y[1345] 	 1.0
Y[1886] 	 1.0
Y[1348] 	 1.0
Y[2523] 	 1.0
Y[3565] 	 1.0
Y[3838] 	 1.0
Y[1542] 	 1.0
Y[1357] 	 1.0
Y[1360] 	 1.0
Y[1365] 	 1.0
Y[2782] 	 1.0
Y[1375] 	 1.0
Y[2094] 	 1.0
Y[1385] 	 1.0
Y[1390] 	 1.0
Y[1393] 	 1.0
Y[2700] 	 1.0
Y[3373] 	 1.0
Y[1394] 	 1.0
Y[1398] 	 1.0
Y[1513] 	 1.0
Y[1781] 	 1.0
Y[1442] 	 1.0
Y[1066] 	 1.0
Y[1408] 	 1.0
Y[1413] 	 1.0
Y[2563] 	 1.0
Y[3700] 	 1.0
Y[1224] 	 1.0
Y[3524] 	 1.0
Y[1436] 	 1.0
Y[2418] 	 1.0
Y[1446] 	 1.0
Y[1327] 	 1.0
Y[1458] 

# LP Rounding

In [14]:
def copy_graph(graph):
    tmpfile = '.copy.bin'

    # Saving to tmp file
    FOut = snap.TFOut(tmpfile)
    graph.Save(FOut)
    FOut.Flush()

    # Loading to new graph
    FIn = snap.TFIn(tmpfile)
    graphtype = type(graph)
    new_graph = graphtype.New()
    new_graph = new_graph.Load(FIn)

    return new_graph

In [15]:
import random
timer = 10
count = 0
print("LP Rounding")
for i in range(timer):
    print("round=",i)
    random_strategy_choose = [] #新策略(格式很亂)
    new_strategy = [[]for i in range(layer)]#新策略(格式清理過)

    for i in range(1,layer+1): #以每一個時間抓出來算機率
        XX = []
        XX_ori = []
        XX_Pro = []

        for x in range(len(THE_X)):
            if THE_X[x][-4]==",":
                if THE_X[x][-3:-1]==str(i):
                    XX.append(THE_X[x])
                    XX_Pro.append(X_Pro[x])
            elif THE_X[x][-2]==str(i):
                XX.append(THE_X[x])
                XX_Pro.append(X_Pro[x])
        if(len(XX)==0): #如果在某時沒有注射
            continue
        XX_ori = XX
#         print("XX_ori",len(XX_ori))
#         print("XX",XX)
#         print("XX_Pro",XX_Pro)
        for i in range(B): #每一個時間挑B個
            if(len(XX_ori)<B):
                random_strategy_choose.append(XX)
                break
            else:
                rdc = random.choices(XX, weights=XX_Pro,k=1) #這次的策略
#                 print("rdc",rdc)
                del XX_Pro[XX.index(rdc[0])]
                XX.remove(rdc[0])

                random_strategy_choose.append(rdc)
#         print("strategy",random_strategy_choose)
    #清理格式    
    for i in range(len(random_strategy_choose)):
        if(random_strategy_choose[i][0][-4]==","): #注射時間是十位數
            new_strategy[int(random_strategy_choose[i][0][-3:-1])-1].append(random_strategy_choose[i][0][2:-4])
        else: #注射時間是個位數
            new_strategy[int(random_strategy_choose[i][0][-2])-1].append(random_strategy_choose[i][0][2:-3])

#     new_strategy = list(map(int, new_strategy)) #str-->int
    print("選擇的策略=",new_strategy)
    ########################################################################開始注射策略
    G_temp = copy_graph(G1)
    S = init_S #初始感染點
    temp_vaccinated = []
    temp_infected = [S]
    temp_vulnerable = V.copy()    
    for i in range(len(new_strategy)): #開始依序注射
        if(len(new_strategy[i])==0):#如果這個時間點沒有注射
            for n in range(len(a)):#病毒開始擴散
                if(G_temp.GetShortPath(S,a[n])==i+1):
                    temp_infected.append(a[n])
                    temp_vulnerable.remove(a[n])
            continue
        TT_saved = []
        ttemp_saved = []
        for j in range(len(new_strategy[i])):
#             temp_saved = []
            b = temp_vulnerable.copy()
            if(int(new_strategy[i][j]) not in temp_vulnerable): #如果要注射的點已經被保護了(從圖中刪除)
#                 for n in range(len(b)):
#                      if(G_temp.GetShortPath(S,b[n])==i+1):#病毒開始擴散
#                         temp_infected.append(b[n])
#                         temp_vulnerable.remove(b[n])
                continue
                

            for k in range(len(temp_vulnerable)):#每一個點計算是否有被注射的點保護

                r  = i+1
                if(r<=(G_temp.GetShortPath(S,temp_vulnerable[k])-G_temp.GetShortPath(int(new_strategy[i][j]),temp_vulnerable[k]))):
                    ttemp_saved.append(temp_vulnerable[k])
        
        temp_saved = list(dict.fromkeys(ttemp_saved))
        for m in range(len(temp_saved)): #調整位置
            temp_vaccinated.append(temp_saved[m])
            temp_vulnerable.remove(temp_saved[m])
            G_temp.DelNode(temp_saved[m])

        a = temp_vulnerable.copy()
        for n in range(len(a)):#病毒開始擴散
            if(G_temp.GetShortPath(S,a[n])==i+1):
                temp_infected.append(a[n])
                temp_vulnerable.remove(a[n])

            
    print("保護個數=",len(temp_vaccinated))
    count = count+int(len(temp_vaccinated))
#     print("被保護的點=",temp_vaccinated)
print("平均被保護個數=",count/timer)            

LP Rounding
round= 0
選擇的策略= [['14', '1744'], ['2938', '1073'], ['372', '46'], ['734', '1038'], ['2285', '3138'], ['1', '815'], ['886', '3443'], ['4', '667'], ['18', '2089'], ['3841', '3317'], ['1544', '3089'], [], []]
保護個數= 2887
round= 1
選擇的策略= [['14', '2543'], ['1073', '180'], ['372', '46'], ['1038', '734'], ['3138', '2285'], ['815', '1'], ['3443', '886'], ['667', '4'], ['18', '2089'], ['3317', '3841'], ['1544', '3089'], [], []]
保護個數= 3889
round= 2
選擇的策略= [['14', '2543'], ['2938', '2082'], ['372', '46'], ['734', '1038'], ['2285', '3138'], ['815', '1'], ['3443', '886'], ['4', '667'], ['2089', '18'], ['3317', '3841'], ['3089', '1544'], [], []]
保護個數= 2884
round= 3
選擇的策略= [['2543', '1744'], ['2082', '2938'], ['46', '372'], ['1038', '734'], ['2285', '3138'], ['815', '1'], ['3443', '886'], ['667', '4'], ['18', '2089'], ['3841', '3317'], ['3089', '1544'], [], []]
保護個數= 3888
round= 4
選擇的策略= [['1744', '2543'], ['2082', '180'], ['372', '46'], ['734', '1038'], ['2285', '3138'], ['1', '815'], ['8